# 🌾 Rice Classification | Part 2: Model Implementation
**Arborio vs Jasmine — Binary Classification**

**Prerequisite:** Run Part 1 first and ensure `part1_data.pkl` is in your Colab session.

**Models built here:** Logistic Regression · Random Forest · SVM · KNN

In [ ]:
# ── Cell 1: Imports ─────────────────────────────────────────────
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle, time, warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model   import LogisticRegression
from sklearn.ensemble       import RandomForestClassifier
from sklearn.svm            import SVC
from sklearn.neighbors      import KNeighborsClassifier
from sklearn.metrics        import accuracy_score, classification_report

sns.set_theme(style='whitegrid', palette='Set2')
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
print('✅ Imports ready')

In [ ]:
# ── Cell 2: Load Preprocessed Data from Part 1 ──────────────────
with open('part1_data.pkl', 'rb') as f:
    d = pickle.load(f)

X_train, X_val, X_test = d['X_train'], d['X_val'], d['X_test']
y_train, y_val, y_test = d['y_train'], d['y_val'], d['y_test']
feature_columns        = d['feature_columns']

print(f'Train: {X_train.shape} | Val: {X_val.shape} | Test: {X_test.shape}')

In [ ]:
# ── Cell 3: Evaluation Helper ────────────────────────────────────
results = {}  # Stores all model results for the final comparison

def report(name, model, X_v, y_v, t):
    """Evaluate model on validation set and store results."""
    preds  = model.predict(X_v)
    acc    = accuracy_score(y_v, preds)
    results[name] = {'model': model, 'val_acc': acc, 'preds': preds, 'time': t}
    print(f'\n{"="*50}')
    print(f'  {name}')
    print(f'  Val Accuracy : {acc*100:.2f}%   |   Train time: {t:.3f}s')
    print(classification_report(y_v, preds, target_names=['Arborio','Jasmine']))

print('✅ Helper ready')

---
## 2.1 — Baseline: Logistic Regression
Draws a straight boundary between the two classes. Fast and interpretable — our performance floor.

In [ ]:
# ── Cell 4: Tune C (regularisation strength inverse) ────────────
# Small C = stronger regularisation (simpler model)
# Large C = weaker regularisation (more complex model, risk of overfitting)

print(f'{"C":<10} {"Val Acc":>10} {"Train Acc":>11}')
print('-' * 34)

lr_tuning = []
for C in [0.01, 0.1, 1.0, 10.0, 100.0]:
    m = LogisticRegression(C=C, max_iter=1000, random_state=RANDOM_SEED)
    m.fit(X_train, y_train)
    tr = accuracy_score(y_train, m.predict(X_train))
    va = accuracy_score(y_val,   m.predict(X_val))
    lr_tuning.append({'C': C, 'val': va, 'train': tr, 'model': m})
    print(f'C={C:<7} Val: {va*100:>6.2f}%   Train: {tr*100:>6.2f}%')

best_lr  = max(lr_tuning, key=lambda r: r['val'])
BEST_C   = best_lr['C']
print(f'\n🏆 Best C: {BEST_C}')

In [ ]:
# ── Cell 5: Plot LR Tuning Curve ────────────────────────────────
fig, ax = plt.subplots(figsize=(8, 4))
c_labels  = [str(r['C'])    for r in lr_tuning]
train_acc = [r['train']*100 for r in lr_tuning]
val_acc   = [r['val']*100   for r in lr_tuning]

ax.plot(c_labels, train_acc, 'o-', label='Train', color='#4C72B0', linewidth=2)
ax.plot(c_labels, val_acc,   's-', label='Val',   color='#DD8452', linewidth=2)
ax.fill_between(c_labels, train_acc, val_acc, alpha=0.1, color='red', label='Gap')
ax.set_xlabel('C Value'); ax.set_ylabel('Accuracy (%)')
ax.set_title('Logistic Regression — Tuning Curve', fontweight='bold')
ax.legend()
plt.tight_layout(); plt.show()

In [ ]:
# ── Cell 6: Train Final LR Model ────────────────────────────────
t0 = time.time()
final_lr = LogisticRegression(C=BEST_C, max_iter=1000, random_state=RANDOM_SEED)
final_lr.fit(X_train, y_train)
report('Logistic Regression (Baseline)', final_lr, X_val, y_val, time.time()-t0)

# Feature coefficients — positive pushes toward Jasmine, negative toward Arborio
coefs = pd.Series(final_lr.coef_[0], index=feature_columns).sort_values(key=abs, ascending=False)
fig, ax = plt.subplots(figsize=(10, 4))
ax.barh(coefs.index, coefs.values,
        color=['#DD8452' if v>0 else '#4C72B0' for v in coefs.values])
ax.axvline(0, color='black', linewidth=0.8, linestyle='--')
ax.set_title('LR Feature Coefficients | Orange→Jasmine  Blue→Arborio', fontweight='bold')
plt.tight_layout(); plt.show()

---
## 2.2a — Advanced Model 1: Random Forest
Ensemble of many Decision Trees. Handles non-linear patterns. Gives feature importance scores.

In [ ]:
# ── Cell 7: Tune Random Forest ───────────────────────────────────
# n_estimators : number of trees (more = stabler, slower)
# max_depth    : how deep each tree can grow (None = unlimited)
# max_features : features considered at each split ('sqrt' is classic default)

rf_configs = [
    {'n_estimators': 50,  'max_depth': 5,    'max_features': 'sqrt'},
    {'n_estimators': 100, 'max_depth': 10,   'max_features': 'sqrt'},
    {'n_estimators': 200, 'max_depth': None, 'max_features': 'sqrt'},
    {'n_estimators': 100, 'max_depth': 10,   'max_features': 'log2'},
    {'n_estimators': 200, 'max_depth': None, 'max_features': 'log2'},
]

rf_tuning  = []
print(f'{"#":<4} {"n_est":>6} {"depth":>7} {"feats":>7} {"Val":>8} {"Train":>9}')
print('-' * 47)

for i, cfg in enumerate(rf_configs):
    m = RandomForestClassifier(**cfg, min_samples_split=5,
                               random_state=RANDOM_SEED, n_jobs=-1)
    m.fit(X_train, y_train)
    tr = accuracy_score(y_train, m.predict(X_train))
    va = accuracy_score(y_val,   m.predict(X_val))
    depth = str(cfg['max_depth']) if cfg['max_depth'] else 'None'
    rf_tuning.append({**cfg, 'val': va, 'train': tr, 'model': m})
    print(f'{i+1:<4} {cfg["n_estimators"]:>6} {depth:>7} {cfg["max_features"]:>7} {va*100:>7.2f}% {tr*100:>8.2f}%')

best_rf = max(rf_tuning, key=lambda r: r['val'])
print(f'\n🏆 Best: n_estimators={best_rf["n_estimators"]}, depth={best_rf["max_depth"]}, feats={best_rf["max_features"]}')

In [ ]:
# ── Cell 8: Train Final RF + Feature Importance ──────────────────
t0 = time.time()
final_rf = RandomForestClassifier(
    n_estimators=best_rf['n_estimators'], max_depth=best_rf['max_depth'],
    max_features=best_rf['max_features'], min_samples_split=5,
    random_state=RANDOM_SEED, n_jobs=-1)
final_rf.fit(X_train, y_train)
report('Random Forest', final_rf, X_val, y_val, time.time()-t0)

importances = pd.Series(final_rf.feature_importances_, index=feature_columns).sort_values()
fig, ax = plt.subplots(figsize=(10, 5))
ax.barh(importances.index, importances.values,
        color=plt.cm.YlOrRd(importances.values / importances.max()))
ax.set_xlabel('Feature Importance (mean impurity reduction)')
ax.set_title('Random Forest — Feature Importance', fontweight='bold')
for bar, val in zip(ax.patches, importances.values):
    ax.text(val+0.001, bar.get_y()+bar.get_height()/2, f'{val*100:.1f}%', va='center', fontsize=9)
plt.tight_layout(); plt.show()

---
## 2.2b — Advanced Model 2: Support Vector Machine (SVM)
Finds the widest margin boundary between classes. RBF kernel handles non-linear patterns.

In [ ]:
# ── Cell 9: Tune SVM ─────────────────────────────────────────────
# kernel : 'linear' = straight boundary | 'rbf' = curved boundary
# C      : regularisation (same concept as LR)
# gamma  : how far each training point's influence reaches ('scale' = safe auto-setting)

svm_configs = [
    {'kernel': 'linear', 'C': 0.1,  'gamma': 'scale'},
    {'kernel': 'linear', 'C': 1.0,  'gamma': 'scale'},
    {'kernel': 'rbf',    'C': 1.0,  'gamma': 'scale'},
    {'kernel': 'rbf',    'C': 10.0, 'gamma': 'scale'},
    {'kernel': 'rbf',    'C': 10.0, 'gamma': 'auto'},
]

svm_tuning = []
print(f'{"#":<4} {"kernel":>8} {"C":>6} {"gamma":>7} {"Val":>8} {"Train":>9}')
print('-' * 48)

for i, cfg in enumerate(svm_configs):
    m = SVC(**cfg, probability=True, random_state=RANDOM_SEED)
    m.fit(X_train, y_train)
    tr = accuracy_score(y_train, m.predict(X_train))
    va = accuracy_score(y_val,   m.predict(X_val))
    svm_tuning.append({**cfg, 'val': va, 'train': tr, 'model': m})
    print(f'{i+1:<4} {cfg["kernel"]:>8} {cfg["C"]:>6} {cfg["gamma"]:>7} {va*100:>7.2f}% {tr*100:>8.2f}%')

best_svm = max(svm_tuning, key=lambda r: r['val'])
print(f'\n🏆 Best: kernel={best_svm["kernel"]}, C={best_svm["C"]}, gamma={best_svm["gamma"]}')

In [ ]:
# ── Cell 10: Train Final SVM ─────────────────────────────────────
t0 = time.time()
final_svm = SVC(kernel=best_svm['kernel'], C=best_svm['C'],
                gamma=best_svm['gamma'], probability=True, random_state=RANDOM_SEED)
final_svm.fit(X_train, y_train)
report('Support Vector Machine', final_svm, X_val, y_val, time.time()-t0)

sv = final_svm.n_support_
print(f'\nSupport vectors — Arborio: {sv[0]} | Jasmine: {sv[1]} | Total: {sum(sv)}')
print('(Fewer support vectors = wider, more confident margin)')

---
## 2.2c — Advanced Model 3: K-Nearest Neighbors (KNN)
No training phase — classifies by majority vote from K most similar training points.

In [ ]:
# ── Cell 11: Tune KNN ────────────────────────────────────────────
# n_neighbors (K) : how many neighbors vote  — small K=overfit, large K=underfit
# weights         : 'uniform'=equal vote | 'distance'=closer neighbors vote more

knn_tuning = []
print(f'{"K":>4} {"weights":>10} {"Val":>8} {"Train":>9}')
print('-' * 35)

for k in [1, 3, 5, 7, 11, 15, 21, 31, 51]:
    for w in ['uniform', 'distance']:
        m = KNeighborsClassifier(n_neighbors=k, weights=w, metric='euclidean')
        m.fit(X_train, y_train)
        tr = accuracy_score(y_train, m.predict(X_train))
        va = accuracy_score(y_val,   m.predict(X_val))
        knn_tuning.append({'k': k, 'w': w, 'val': va, 'train': tr, 'model': m})
        print(f'K={k:<3} {w:>10} {va*100:>7.2f}% {tr*100:>8.2f}%')

best_knn = max(knn_tuning, key=lambda r: r['val'])
print(f'\n🏆 Best: K={best_knn["k"]}, weights={best_knn["w"]}')

In [ ]:
# ── Cell 12: KNN Tuning Curve (Bias-Variance Tradeoff) ──────────
for weight_type in ['uniform', 'distance']:
    subset = [r for r in knn_tuning if r['w'] == weight_type]
    k_vals = [r['k']        for r in subset]
    tr_acc = [r['train']*100 for r in subset]
    va_acc = [r['val']*100   for r in subset]

    fig, ax = plt.subplots(figsize=(9, 4))
    ax.plot(k_vals, tr_acc, 'o-', label='Train', color='#4C72B0', linewidth=2)
    ax.plot(k_vals, va_acc, 's-', label='Val',   color='#DD8452', linewidth=2)
    ax.fill_between(k_vals, tr_acc, va_acc, alpha=0.1, color='red')
    best_k = max(subset, key=lambda r: r['val'])['k']
    ax.axvline(best_k, color='green', linestyle='--', linewidth=1.5, label=f'Best K={best_k}')
    ax.set_xlabel('K'); ax.set_ylabel('Accuracy (%)')
    ax.set_title(f'KNN Tuning — {weight_type} weights | Small K=overfit, Large K=underfit', fontweight='bold')
    ax.legend(); ax.set_xticks(k_vals)
    plt.tight_layout(); plt.show()

In [ ]:
# ── Cell 13: Train Final KNN ─────────────────────────────────────
t0 = time.time()
final_knn = KNeighborsClassifier(n_neighbors=best_knn['k'],
                                  weights=best_knn['w'], metric='euclidean')
final_knn.fit(X_train, y_train)
report('K-Nearest Neighbors', final_knn, X_val, y_val, time.time()-t0)

In [ ]:
# ── Cell 14: Model Comparison Table ──────────────────────────────
comp = pd.DataFrame([
    {'Model': name, 'Val Accuracy (%)': round(r['val_acc']*100, 2), 'Time (s)': round(r['time'], 3)}
    for name, r in results.items()
]).sort_values('Val Accuracy (%)', ascending=False).reset_index(drop=True)
comp.index += 1

print('\n📊 MODEL COMPARISON (Validation Set)')
print('=' * 55)
print(comp.to_string())
print('=' * 55)

fig, axes = plt.subplots(1, 2, figsize=(13, 5))
names  = comp['Model'].tolist()
accs   = comp['Val Accuracy (%)'].tolist()
times  = comp['Time (s)'].tolist()
colors = ['#2ecc71' if i==0 else '#3498db' for i in range(len(names))]

bars = axes[0].barh(names[::-1], accs[::-1], color=colors[::-1], edgecolor='white')
axes[0].set_title('Validation Accuracy', fontweight='bold')
axes[0].set_xlim(min(accs)-2, 101)
for bar, v in zip(bars, accs[::-1]):
    axes[0].text(v+0.1, bar.get_y()+bar.get_height()/2, f'{v:.2f}%', va='center', fontweight='bold')

axes[1].barh(names[::-1], times[::-1], color='#9b59b6', edgecolor='white')
axes[1].set_title('Training Time (s)', fontweight='bold')
plt.suptitle('Part 2 Summary: All Models | 🟢 = Best accuracy', fontweight='bold', y=1.02)
plt.tight_layout(); plt.show()

In [ ]:
# ── Cell 15: Save All Models for Part 3 ──────────────────────────
import pickle

out = {
    'X_train': X_train, 'X_val': X_val, 'X_test': X_test,
    'y_train': y_train,  'y_val': y_val,  'y_test': y_test,
    'feature_columns': feature_columns,
    'models': {
        'Logistic Regression':     final_lr,
        'Random Forest':           final_rf,
        'Support Vector Machine':  final_svm,
        'K-Nearest Neighbors':     final_knn,
    },
    'val_results': results
}

with open('part2_models.pkl', 'wb') as f:
    pickle.dump(out, f)

print('💾 Saved to part2_models.pkl')
print('   → Open Part 3 notebook next')